# Creating a Collaborative Closing Transaction

In this section, we'll build a Lightning channel closing transaction from scratch using Python. We'll walk through each part of the transaction — how it's constructed, signed, and the messages exchanged between peers to coordinate the close. The process will be tested using Bitcoin Core in regtest mode.

## Setup

For this notebook, we’ll use the Chapter 1 – Channel Funding Transaction, where the funding transaction was created and confirmed on our regtest blockchain.

In [1]:
# run notebook
%run "/home/pins-dev/Projects/Taproot-Lightning-Channels-Workshop/chapter 1 - channel funding transaction/funding-transaction.ipynb"

2025-09-07T17:42:00.040000Z TestFramework (INFO): PRNG seed is: 7597397052372604182
2025-09-07T17:42:00.042000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_9mimiyz_
Alice funding pubkey: aa98c9adc0711168c1652272c54942e301e9fe53ada7cb0598b5a471da8c148c
Alice funding privkey: ef3bef5ca0304922311bf8679c4f47760198a08f4c46c95fd4ae9a426d199de2
Alice funding address: bcrt1p42vvntwqwygk3st9yfev2j2zuvq7nljn4knukpvckkj8rk5vzjxqja98td
Bob funding pubkey: 8da36ea99cd6a7bf02a099cbc3e35daf1683d56ffceda08538f4424e80502d27
Bob funding privkey: 9f975c258521f096a54ac3389fba9dd21f2ec522be3b8c82c77ef2491b4ac51d
Bob funding address: bcrt1p3k3ka2vu66nm7q4qn89u8c6a4utg84t0lnk6ppfc73pyaqzs95nsrsxy66
Transaction creating Alice UTXO: f55d8bbf6a57bb312440013757f4c2ba791ccdd4713d50120030e49bf3685738
{
  "txid": "f55d8bbf6a57bb312440013757f4c2ba791ccdd4713d50120030e49bf3685738",
  "hash": "742a0db8ef1888f1145cfbad61e9cc70a4b413633acd75bba92eda44067f77ca",
  "version": 2,
  "size": 289,

## Legacy cooperative closure

Compared to the base segwit v0 channel type, for simple taproot channels, the co-op close transaction now always signals RBF. In other words, the sequence field of the sole input to the cooperative close transaction MUST be less-than-or-equal to 0xfffffffd. This enables a future cooperative closure flow to support increasing the fee of subsequent close offers via RBF.

In addition, rather than adopt the existing cooperative closure fee rate "negotiation", the responder SHOULD now always accept the offer sent by the initiator. In other words, the cooperative close process now terminates after exactly 1 RTTs: initiator sends sigs with offer, with the responder echo'ing back the same fee rate. This serves to ensure that the co-op close process always terminates deterministically, and also plays nicer with the nonces: only a single message is ever signed by both sides for a coop close workflow.

    +-----------+                                        +---------+
    |           |--(1)-------- shutdown_nonce ---------->|         |
    |           |<-(2)-------- shutdown_nonce -----------|         |
    |           |                                        |         |
    |   Alice   |--(3)-- closing_signed (partial_sig) -->|   Bob   |
    |           |<-(4)-- closing_signed (partial_sig) ---|         |
    |           |                                        |         |
    |           |                                        |         |
    |           |                                        |         |
    +-----------+                                        +---------+

## Modern RBF-based cooperative close

For taproot channels supporting RBF cooperative close, nonces are delivered just-in-time (JIT) with signatures using an asymmetric pattern:

Initial Exchange: Nonces are exchanged in shutdown messages

Each party sends their "closee nonce" - the nonce they'll use when signing as the closee. This nonce is used by the remote party when they act as closer.

#### RBF Iterations: Subsequent nonces are delivered with signatures

* closing_complete (from closer): uses partial_sig_w_nonce bundles the partial signature with the sender's closer nonce. The bundling is necessary because the closee doesn't know this nonce yet.

* closing_sig (from closee): uses partial_sig plus nonce (NextCloseeNonce) field

Asymmetric Roles:

* Closer: The party sending closing_complete, proposing a new fee
* Closee: The party sending closing_sig, accepting the fee proposal

Each party alternates between these roles during RBF iterations.

    +-----------+                                                +---------+
    |           |--(1)------------- shutdown_nonce ------------->|         |
    |           |<-(2)------------- shutdown_nonce --------------|         |
    |           |                                                |         |
    |   Alice   |--(3)- closing_complete (partial_sig_w_nonce) ->|   Bob   |
    |           |<-(4)---- closing_sig (partial_sig) (nonce)-----|         |
    |           |                                                |         |
    |           |<-(5)- closing_complete (partial_sig_w_nonce) ->|         |
    |           |--(6)---- closing_sig (partial_sig) (nonce)---->|         |
    |           |                                                |         |
    +-----------+                                                +---------+



## The Unsined Transaxaction

In [2]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(funding_channel_txid))[::-1]
funding_channel_index = 0
index = funding_channel_index.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# Use `0xfffffffd` to enable a future rbf cooperative closure round
sequence = bytes.fromhex("fdff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# Transaction Fee
tx_fee_sat = int(float("0.00000300") * 100000000)

# OUTPUT 1 - Back to Alice
# 0.005 BTC discounted the fees
# In the legacy cooperative closure the channel founder pay the fees
# In the new rbf cooperative closure the closer pay the fees
output1_value_sat = int(float("0.005") * 100000000) - tx_fee_sat
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
output1_spk = bytes.fromhex("51") + varint_len(alice_funding_pubkey_b) + alice_funding_pubkey_b

# OUTPUT 2 - Back to Bob
# 0.005 BTC
output2_value_sat = int(float("0.005") * 100000000)
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
output2_spk = bytes.fromhex("51") + varint_len(bob_funding_pubkey_b) + bob_funding_pubkey_b

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

unsigned_tx:  0200000001f29d9e5208888541b765a8b2443161baec5f47e7d4b040912e9f933b545ae5b60000000000fdffffff02f49f070000000000225120aa98c9adc0711168c1652272c54942e301e9fe53ada7cb0598b5a471da8c148c20a10700000000002251208da36ea99cd6a7bf02a099cbc3e35daf1683d56ffceda08538f4424e80502d2700000000
{
  "txid": "d2f956c422f9b14a7f8429b302407bbdff7126452cb07c43729e4c67b67b03a5",
  "hash": "d2f956c422f9b14a7f8429b302407bbdff7126452cb07c43729e4c67b67b03a5",
  "version": 2,
  "size": 137,
  "vsize": 137,
  "weight": 548,
  "locktime": 0,
  "vin": [
    {
      "txid": "b6e55a543b939f2e9140b0d4e7475fecba613144b2a865b741858808529e9df2",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967293
    }
  ],
  "vout": [
    {
      "value": "0.00499700",
      "n": 0,
      "scriptPubKey": {
        "asm": "1 aa98c9adc0711168c1652272c54942e301e9fe53ada7cb0598b5a471da8c148c",
        "desc": "rawtr(aa98c9adc0711168c1652272c54942e301e9fe53ada7cb0598b5a471

## The sighash for the key path spend

This is the “Schnorr key spend” case: you’re proving knowledge of the (tweaked) internal private key, with no script branch revealed. The message preimage is called msg_digest in [BIP-341](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki).

In [3]:
index_of_this_input = bytes.fromhex("0000 0000")

# SIGHASH for key path spend
# See BIP-341 for details
sighash_epoch = bytes.fromhex("00")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index)

input_amounts = channel_value
sha_amounts = sha256(input_amounts)

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
sha_scriptpubkeys = sha256(
    varint_len(channel_spk)
    + channel_spk
)

sha_sequences = sha256(sequence)

sha_outputs = sha256(outputs) 

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode())
sighash = sha256(tag_hash + tag_hash + sig_msg)


## Signing the sighash

In [4]:
# Alice and Bob generates its own nonce pair (secnonce, pubnonce)
alice_secnonce1, alice_pubnonce1 = nonce_gen(privkey_alice_musig2.get_bytes(), pubkey_alice_musig2, agg_pubkey_tweaked, sighash, None)
bob_secnonce1, bob_pubnonce1 = nonce_gen(privkey_bob_musig2.get_bytes(), pubkey_bob_musig2, agg_pubkey_tweaked, sighash, None)

print(f"Alice secnonce: {alice_secnonce1.hex()} and pubnonce: {alice_pubnonce1.hex()}")
print(f"Bob secnonce: {bob_secnonce1.hex()} and pubnonce: {bob_pubnonce1.hex()}")

# Alice and Bob has exchanged already the pubnonces, so they can create agregate them and create the session context
agg_nonce = nonce_agg([alice_pubnonce1, bob_pubnonce1])
session = SessionContext(agg_nonce, sorted_pubkeys, [bip86_tweak], [True], sighash)

# Alice and Bob produces their partial signatures
alice_psig = sign(alice_secnonce1, privkey_alice_musig2.get_bytes(), session)
bob_psig = sign(bob_secnonce1, privkey_bob_musig2.get_bytes(), session)

# Each side verify the other’s partial signature before proceeding
assert partial_sig_verify(alice_psig, [alice_pubnonce1, bob_pubnonce1], sorted_pubkeys, [bip86_tweak], [True], sighash, 0)
assert partial_sig_verify(bob_psig, [alice_pubnonce1, bob_pubnonce1], sorted_pubkeys, [bip86_tweak], [True], sighash, 1)

# Each side combines partial signatures into the final Schnorr signature
agg_sig = partial_sig_agg([alice_psig, bob_psig], session)  # r||s, 64 bytes

# Sanity check: verify with BIP340 against the *tweaked* x-only key ---
ok = schnorr_verify(sighash, agg_pubkey_tweaked, agg_sig)
print("aggregated Schnorr verifies?", ok)


Alice secnonce: 2cb68ab4665deb808d266580563446575bf303de769cd8452e9c39becb3cb124c67d2da76decc5ae9a534f67c3314bac50129d8512038d84d4b9e0004c4867a30282264267172144b0b6912b788e33c21563cd1194c4c6565dfc04495b98976478 and pubnonce: 02509df95aa8916468a2c0f8bc38c0b2b4b79d2693652a9d746525adf2dcd53209031a3e49cd8764cb079bd577d976d05fef3d7a7dff4e2f92128a495226d80975d8
Bob secnonce: ddafc3fecf752c20bd61c8a3956bdc3fbc943384efcebe9734aee97d101d76cd33b82b527472b64897abbdf7d07a9da84b79436c8c02e18deca3906dd093015202ea091f3a152bd83704660fd3ea0f3cc9b057429c6d24e553539214ff8d8cb4f2 and pubnonce: 030a10ee0efde6e1c38fa7a7a5afb1633991b566cdb18cacab98ab884fad83b39d02bad007b67a0589d7c84c19c76dfda8e367ffc00899cc1530cc4d5baae453192e
aggregated Schnorr verifies? True
